In [1]:
import os
import yaml
import torch

import sys
# cur_dir = os.path.split(os.path.abspath(__file__))[0]
# con_pth = cur_dir.rsplit('/',2)[0]
# sys.path.append(con_pth)
from src.models.celltrack_plmodel import CellTrackLitModel
from src.inference.graph_dataset_inference import CellTrackDataset
import warnings
# warnings.filterwarnings("ignore")

import os
import os.path as osp
import torch
import pandas as pd
import numpy as np
from skimage import io
import warnings
# warnings.filterwarnings("ignore")
import imageio

/home/sunrui/anaconda3/envs/cell-tracking-challenge/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sunrui/anaconda3/envs/cell-tracking-challenge/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/sunrui/anaconda3/envs/cell-tracking-challenge/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c106IValue23reportToTensorTypeErrorEv
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model_path = "/home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04/checkpoints/epoch=56.ckpt"
num_seq = "01"
output_csv = '/data/sunrui/celldata/r03c06f04ch1/'

path_output = output_csv
second_path = num_seq
path_output_folder = os.path.join(path_output, second_path) + '_RES_inference'

In [3]:
file1 = os.path.join(path_output_folder, 'pytorch_geometric_data.pt')
file2 = os.path.join(path_output_folder, 'all_data_df.csv')
file3 = os.path.join(path_output_folder, 'raw_output.pt')

In [4]:
df_data = pd.read_csv(file2)
test_data = torch.load(file1)
outputs = torch.load(file3)

In [5]:
def find_connected_edges():
    edge_index, df, outputs = edge_index, df_preds, output_pred

    if not self.directed:
        final_outputs_hard, edge_index = self.megre_match_edges(edge_index.detach().clone(), outputs.detach().clone())
        self.outputs_hard = final_outputs_hard
        self.edge_index = edge_index
    else:
        outputs_soft = torch.sigmoid(outputs)
        self.outputs_hard = (outputs_soft > self.decision_threshold).int()
# find_connected_edges()

In [6]:
print(test_data.edge_index)
# i = 0
# connected_indices = test_data.edge_index
# ind_place = np.argwhere(connected_indices[0, :] == 0)
# print(ind_place)
# next_frame_ind = connected_indices[1, ind_place].numpy().squeeze()
# print(next_frame_ind)
# next_frame = df_preds.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
# curr_node = df_preds.loc[i,["centroid_row", "centroid_col"]].values
# distance = ((next_frame - curr_node) ** 2).sum(axis=-1)
# nearest_cell = np.argmin(distance, axis=-1)
# # add to the array
# next_node_ind = next_frame_ind[nearest_cell]
# print(next_node_ind)

# next_node_ind = connected_indices[1, ind_place[0]]
# ind_place = ind_place
# print(ind_place)
# next_node_ind = connected_indices[1, ind_place[0]]
# print(next_node_ind)

tensor([[    0,     0,     0,  ..., 16146, 16146, 16146],
        [   53,    55,    74,  ..., 16154, 16164, 16171]])


In [7]:
print(outputs)

tensor([ 18.4344, -20.9180, -16.0451,  ...,  -3.7429, -11.5620,  17.1256])


In [8]:
# print(edge_index)
# print(edge_index[:, ::2])
# print(edge_index[[1, 0], 1::2])

In [15]:
data = test_data
edge_index = data.edge_index
df_preds = df_data
output_pred = outputs
decision_threshold = 0.5
df = df_preds.copy()

In [20]:
outputs_soft = torch.sigmoid(output_pred)
outputs_hard = (outputs_soft > decision_threshold).int()
print(outputs_hard)

connected_indices = edge_index[:, outputs_hard.bool()]#取出符合要求的边缘
output_save_pre = output_pred[outputs_hard.bool()]
print(connected_indices)#change
print(connected_indices.shape)

label_fir = connected_indices[0,:]
label_sec = connected_indices[1,:]
label_sec_unique = np.unique(label_sec)


# 获取每个标签的行和列
first_positions = df.loc[label_fir, ["centroid_row", "centroid_col"]].values
second_positions = df.loc[label_sec, ["centroid_row", "centroid_col"]].values

# 计算距离矩阵
distances = ((second_positions - first_positions) ** 2).sum(axis=-1)
distances = np.sqrt(distances)

# 创建一个矩阵来标识需要保留的连接
keep_mask = np.zeros_like(distances, dtype=bool)


for label in label_sec_unique:
        label_indices = np.where(label_sec == label)[0]
        if len(label_indices) > 1:
            label_distances = distances[label_indices]
            min_distance_index = np.argmin(label_distances)
            keep_mask[label_indices[min_distance_index]] = True
        else:
            keep_mask[label_indices] = True
        
# 使用布尔索引删除不需要的连接
connected_indices = connected_indices[:, keep_mask]
output_save_pre = output_save_pre[keep_mask]

# for label in label_sec_unique:
#     label_cols = label_sec[label_sec == label]
#     if label_cols.shape[0] > 1:
      
#         ind_place_col = np.argwhere(label_sec==label)
      
#         curr_position = df.loc[label,["centroid_row", "centroid_col"]].values   #第二行的位置
        
        
#         x = label_fir[ind_place_col].numpy().squeeze()
#         first_position = df.loc[x, ["centroid_row", "centroid_col"]].values     #第一行的位置

#         distance = ((curr_position - first_position) ** 2).sum(axis=-1)
#         nearest_cell = np.argmin(distance, axis=-1)
#         node_remain = x[nearest_cell]     #要保留的ID


#         delet_id = np.delete(x, np.where(x == node_remain))#需要改数据类型

#         for del_id in delet_id:
#             # 使用布尔索引删除连接
#             delet_mask = np.logical_and(connected_indices[0, :] == del_id, connected_indices[1, :] == label)
#             delete_indices = np.where(delet_mask)[0]
#             delete_indices_test = np.where(delet_mask)
#             connected_indices = np.delete(connected_indices, delete_indices, axis=1)


def change_sec_id(self):


    

    

   

    # 打印结果
    print("Connected Indices (After Filtering):")
    print(connected_indices)
    print(connected_indices.shape)
    print(output_save_pre)
    print(output_save_pre.shape)

    self.output_pred = output_save_pre
    self.edge_index = connected_indices






# 打印结果
print("Connected Indices (After Filtering):")
print(connected_indices)
print(connected_indices.shape)

      

# next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
# curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values

# distance = ((next_frame - curr_node) ** 2).sum(axis=-1)
# nearest_cell = np.argmin(distance, axis=-1)
# # add to the array
# next_node_ind = next_frame_ind[nearest_cell]

tensor([1, 0, 0,  ..., 0, 0, 1], dtype=torch.int32)
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
torch.Size([2, 20012])
Connected Indices (After Filtering):
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
torch.Size([2, 15373])


In [26]:
# 使用断言确保 edge_index 的形状匹配
# assert torch.all(edge_index[:, ::2] == edge_index[[1, 0], 1::2]), "The results don't match!"

# 对 edge_index 进行处理，仅保留一部分
edge_index = edge_index[:, ::2]

# 从 output_pred 中提取输入和输出的预测结果
in_output_pred = output_pred[::2]
out_output_pred = output_pred[1::2]

# if self.merge_operation == 'OR' or self.merge_operation == 'AND':
# 使用 sigmoid 函数对预测结果进行软化
in_outputs_soft = torch.sigmoid(in_output_pred)
in_outputs_hard = (in_outputs_soft > decision_threshold).int()

out_outputs_soft = torch.sigmoid(out_output_pred)
out_outputs_hard = (out_outputs_soft > decision_threshold).int()

# 根据合并操作进行逻辑运算（OR 或 AND）
# final_outputs_hard_or = np.bitwise_or(in_outputs_hard, out_outputs_hard) #if self.merge_operation == 'OR' \
# final_outputs_hard_or_and = np.bitwise_and(in_outputs_hard, out_outputs_hard)
print(in_outputs_hard)
print(out_outputs_hard)

tensor([1, 0, 0,  ..., 0, 0, 1], dtype=torch.int32)
tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)


In [16]:
CKPT_PATH = model_path
path_output = output_csv
folder_path = CKPT_PATH
for i in range(2):
    folder_path = folder_path[:folder_path.rfind('/')]
    print(folder_path)
config_path = os.path.join(folder_path, '.hydra/config.yaml')
print('config_path:', config_path)
config = yaml.load(open(config_path),Loader=yaml.Loader)
# print(type(config))
# print(config.items())
for key,value in config.items():
    print('key:',key,'value:',value)

print('..............................................')


data_yaml = config['datamodule']

for key,value in data_yaml.items():
    print('key:',key,'value:',value)

/home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04/checkpoints
/home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04
config_path: /home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04/.hydra/config.yaml
key: trainer value: {'_target_': 'pytorch_lightning.Trainer', 'gpus': 1, 'min_epochs': 1, 'max_epochs': 500, 'weights_summary': 'top', 'progress_bar_refresh_rate': 100, 'terminate_on_nan': True}
key: model value: {'_target_': 'src.models.celltrack_plmodel.CellTrackLitModel', 'one_hot_label': '${one_hot}', 'loss_weights': 65, 'sample': False, 'weight_loss': True, 'directed': True, 'separate_models': '${separate_models}', 'optim_module': {'target': 'Adam', 'kwargs': {'lr': 0.001, 'weight_decay': 1e-05}}, 'lr_sch_module': {'target': None, 'monitor': 'val/acc', 'kwargs': {'verbose': True, 'mode': 'min'}}, 'model_params': {'target': 'CellTrack_Model', 'kwargs': {'hand_NodeEncoder_dic': {'input_dim': 13, 'fc_dims

/tmp/ipykernel_51806/913783244.py:9: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04/.hydra/config.yaml' mode='r' encoding='UTF-8'>
  config = yaml.load(open(config_path),Loader=yaml.Loader)


In [17]:
trained_model = CellTrackLitModel.load_from_checkpoint(checkpoint_path=CKPT_PATH)
print(trained_model.hparams)
trained_model.eval()
trained_model.freeze()
print(type(trained_model))

"directed":        True
"loss_weights":    65
"lr":              0.001
"lr_sch_module":   {'target': None, 'monitor': 'val/acc', 'kwargs': {'verbose': True, 'mode': 'min'}}
"model_params":    {'target': 'CellTrack_Model', 'kwargs': {'hand_NodeEncoder_dic': {'input_dim': 13, 'fc_dims': [64, 16]}, 'learned_NodeEncoder_dic': {'input_dim': 128, 'fc_dims': [64, 16]}, 'intialize_EdgeEncoder_dic': {'input_dim': 239, 'fc_dims': [128, 64]}, 'message_passing': {'target': 'CellTrack_GNN', 'kwargs': {'in_channels': 32, 'hidden_channels': 32, 'in_edge_channels': 64, 'hidden_edge_channels_conv': 16, 'hidden_edge_channels_linear': [128, 64], 'dropout': 0.0, 'num_layers': 6, 'num_nodes_features': 3}}, 'edge_classifier_dic': {'input_dim': 64, 'fc_dims': [128, 32, 1], 'dropout_p': 0.2, 'use_batchnorm': False}}}
"one_hot_label":   True
"optim_module":    {'target': 'Adam', 'kwargs': {'lr': 0.001, 'weight_decay': 1e-05}}
"sample":          False
"separate_models": True
"weight_decay":    0.0005
"weight_lo

In [18]:
data_yaml['dataset_params']['num_frames'] = 'all'
data_yaml['dataset_params']['main_path'] = path_output

second_path = num_seq
data_yaml['dataset_params']['dirs_path']['test'] = [second_path + "_CSV"]
test_yaml = data_yaml['dataset_params']

print(test_yaml)
for key,value in test_yaml.items():
    print('key:',key,'value:',value)
dirs_path = test_yaml['dirs_path']
data_train: CellTrackDataset = CellTrackDataset(**data_yaml['dataset_params'], split='test')
data_list, df_list = data_train.all_data['test']
test_data, df_data = data_list[0], df_list[0]

x, x2, edge_index, edge_feature = test_data.x, test_data.x_2, test_data.edge_index, test_data.edge_feat

{'num_frames': 'all', 'type_file': 'csv', 'produce_gt': 'simple', 'main_path': '/data/sunrui/celldata/r03c06f04ch1/', 'dirs_path': {'train': ['01', '02'], 'test': ['01_CSV']}, 'one_hot_label': '${one_hot}', 'exp_name': '2D_SIM', 'overlap': 1, 'jump_frames': 1, 'mul_vals': [2, 2, 2], 'filter_edges': True, 'directed': True, 'edge_feat_embed_dict': {'p': 1, 'use_normalized_x': True, 'normalized_features': False}, 'separate_models': '${separate_models}', 'normalize_all_cols': False, 'same_frame': False, 'next_frame': True, 'self_loop': True, 'normalize': True, 'debug_visualization': False, 'which_preprocess': 'MinMax', 'drop_feat': []}
key: num_frames value: all
key: type_file value: csv
key: produce_gt value: simple
key: main_path value: /data/sunrui/celldata/r03c06f04ch1/
key: dirs_path value: {'train': ['01', '02'], 'test': ['01_CSV']}
key: one_hot_label value: ${one_hot}
key: exp_name value: 2D_SIM
key: overlap value: 1
key: jump_frames value: 1
key: mul_vals value: [2, 2, 2]
key: filt

/home/sunrui/cellwork/track/cell-tracker-gnn-main/src/inference/graph_dataset_inference.py:353: UserWarning: Find the seg label as part of the features and dropped it, please be aware
  warnings.warn("Find the seg label as part of the features and dropped it, please be aware")


features: Index(['frame_num', 'area', 'min_row_bb', 'min_col_bb', 'max_row_bb',
       'max_col_bb', 'centroid_row', 'centroid_col', 'major_axis_length',
       'minor_axis_length',
       ...
       'feat_118', 'feat_119', 'feat_120', 'feat_121', 'feat_122', 'feat_123',
       'feat_124', 'feat_125', 'feat_126', 'feat_127'],
      dtype='object', length=141)
Finish frame index 0
Finish process /data/sunrui/celldata/r03c06f04ch1/01_CSV/csv (test)


In [19]:
import os.path as osp
from collections.abc import Iterable
import pandas as pd
import numpy as np
import itertools
dirs_path = test_yaml['dirs_path']
main_path = test_yaml['main_path']
type_file = test_yaml['type_file']
num_frames = test_yaml['num_frames']
directed = True
same_frame=True
next_frame=True
self_loop=True
filter_edges = False
jump_frames = 1
overlap = 1
mul_vals = [2, 2, 2]
def bb_roi(df_data):
    print('bb_roi')
    cols = ['min_row_bb', 'min_col_bb', 'max_row_bb', 'max_col_bb']

    bb_feat = df_data.loc[:, cols]
    max_row = np.abs(bb_feat.min_row_bb.values - bb_feat.max_row_bb.values).max()
    max_col = np.abs(bb_feat.min_col_bb.values - bb_feat.max_col_bb.values).max()

    curr_roi = {'row': max_row * mul_vals[0], 'col': max_col * mul_vals[1]}
    return curr_roi


def find_roi(files, curr_dir):
    print('find_roi')
    temp_data = [pd.read_csv(file) for file in files]
    df_data = pd.concat(temp_data, axis=0).reset_index(drop=True)
    curr_roi = bb_roi(df_data)
    return curr_roi


def filter_by_roi():
    pass

def same_next_links(df_data, link_edges):
    #doing aggregation of the same frame links + the links between 2 consecutive frames
    global same_frame, self_loop, next_frame

    same_next_edge_index = []
    iter_frames = np.unique(df_data.frame_num.values)
    print(iter_frames)
    for loop_ind, frame_ind in enumerate(iter_frames[:-1]):

        mask_frame = df_data.frame_num.isin([frame_ind])
        nodes = df_data.index[mask_frame].values.tolist()
        # doing aggregation of the same frame links
        if same_frame:
            if self_loop:
                same_next_edge_index += [list(tup) for tup in itertools.product(nodes, nodes)]
            else:
                same_next_edge_index += [list(tup) for tup in itertools.product(nodes, nodes) if tup[0] != tup[1]]
            
            # print(same_next_edge_index)
        if next_frame:
            if frame_ind != iter_frames[-1]:
                # find the places containing the specific frame index
                mask_next_frame = df_data.frame_num.isin([iter_frames[loop_ind + 1]])
                next_nodes = df_data.index[mask_next_frame].values.tolist()
                if filter_edges:
                    curr_list = filter_by_roi(df_data.loc[mask_frame, :], df_data.loc[mask_next_frame, :])
                    curr_list = list(filter(lambda x: not (x in link_edges), curr_list))
                else:
                    curr_list =  [list(tup) for tup in itertools.product(nodes, next_nodes)
                                    if not (list(tup) in link_edges)]
                if not directed:
                    # take the opposite direction using [::-1] and merge one-by-one
                    # with directed and undirected edges
                    curr_list_opposite = [pairs[::-1] for pairs in curr_list]
                    curr_list = list(itertools.chain.from_iterable(zip(curr_list, curr_list_opposite)))
                same_next_edge_index += curr_list
    return same_next_edge_index






def create_graph(curr_dir, mode):
    data_list = []
    df_list = []
    drop_col_list = []
    is_first_time = True
    global num_frames, jump_frames, type_file, same_frame, next_frame
    files = [osp.join(curr_dir, f_name) for f_name in sorted(os.listdir(curr_dir)) if
            type_file in f_name]
    print(f"Start with {curr_dir}")
    num_files = len(files)
    curr_roi = find_roi(files, curr_dir)
    print(curr_roi)
    
    if num_frames == 'all':
        num_frames = num_files
    elif isinstance(num_frames, int):
        num_frames = num_frames

    for ind in range(0, num_files, overlap):
        if ind + num_frames > num_files:
            break
        #read current frame CSVs
        temp_data = [pd.read_csv(files[ind_tmp]) for ind_tmp in range(ind, ind + num_frames, jump_frames)]
        df_data = pd.concat(temp_data, axis=0).reset_index(drop=True)
        print("df_data")
        # for key,value in df_data.items():
        #     print('key:',key,'value:',value)
        #     print('key:',key)
        print(df_data)
        link_edges = []
        if same_frame or next_frame:
            same_next_links(df_data, link_edges)
            # link_edges += same_next_links(df_data, link_edges)
        


    

In [20]:
frame_ind = 0
ind = 0
print(curr_dir)
same_next_edge_index = []
files = [osp.join(curr_dir, f_name) for f_name in sorted(os.listdir(curr_dir)) if
        type_file in f_name]
temp_data = [pd.read_csv(files[ind_tmp]) for ind_tmp in range(ind, ind + num_frames, jump_frames)]
df_data = pd.concat(temp_data, axis=0).reset_index(drop=True)
# print(df_data)
mask_frame = df_data.frame_num.isin([frame_ind])
# print(mask_frame)
nodes = df_data.index[mask_frame].values.tolist()
same_next_edge_index += [list(tup) for tup in itertools.product(nodes, nodes)]

NameError: name 'curr_dir' is not defined

In [ ]:

for k, v_list in dirs_path.items():
    for ind, val in enumerate(v_list):
        dirs_path[k][ind] = osp.join(main_path, val)
print(dirs_path)
split='test'
curr_mode = split
all_data = {}
curr_dir = dirs_path[curr_mode]

print(curr_dir)

if isinstance(curr_dir, str):
    curr_dir = osp.join(curr_dir,type_file)
    print(curr_dir)

elif isinstance(curr_dir, Iterable):
    data_list_t = []
    for dir_path in curr_dir:
        curr_dir = osp.join(dir_path, type_file)
        print('curr_dir:',curr_dir)
        create_graph(curr_dir,curr_mode)
        # data_list_t += create_graph(curr_dir,curr_mode)

print(curr_dir)

{'train': ['/data/sunrui/celldata/r03c06f04ch1/01', '/data/sunrui/celldata/r03c06f04ch1/02'], 'test': ['/data/sunrui/celldata/r03c06f04ch1/01_CSV']}
['/data/sunrui/celldata/r03c06f04ch1/01_CSV']
curr_dir: /data/sunrui/celldata/r03c06f04ch1/01_CSV/csv
Start with /data/sunrui/celldata/r03c06f04ch1/01_CSV/csv
find_roi
bb_roi
{'row': 484, 'col': 276}
df_data
       seg_label  frame_num  area  min_row_bb  min_col_bb  max_row_bb  \
0             13          0  6868         933         220        1023   
1             14          0  6855         674         318         774   
2             15          0  6568         303         133         400   
3             16          0  6489         766         805         872   
4             17          0  6009         202          96         293   
...          ...        ...   ...         ...         ...         ...   
16167         22        328  2341        1030         795        1080   
16168         23        328  1918         840         781  

In [ ]:
outputs = trained_model((x, x2), edge_index, edge_feature.float())
data_path = os.path.join(path_output, second_path) + '_RES_inference'
path_output_folder = data_path
print(f"save path : {path_output_folder}")
os.makedirs(path_output_folder, exist_ok=True)
file1 = os.path.join(path_output_folder, 'pytorch_geometric_data.pt')
file2 = os.path.join(path_output_folder, 'all_data_df.csv')
file3 = os.path.join(path_output_folder, 'raw_output.pt')
print(f"Save inference files: \n - {file1} \n - {file2} \n - {file3}")
# df_data.to_csv(file2)
# torch.save(test_data, file1)
# torch.save(outputs, file3)

save path : /data/sunrui/celldata/r03c06f04ch1/01_RES_inference
Save inference files: 
 - /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/pytorch_geometric_data.pt 
 - /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/all_data_df.csv 
 - /data/sunrui/celldata/r03c06f04ch1/01_RES_inference/raw_output.pt
